In [ ]:
!pip install transformers -qqq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 68.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 32.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 73.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 81.8 MB/s eta 0:00:00


In [ ]:
import torch
from transformers import AutoTokenizer

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
%cd /content/drive/MyDrive/ERA_v1/Session17/dev4

Mounted at /content/drive/
/content/drive/MyDrive/ERA_v1/Session17/dev4


In [ ]:
from transformer_model.models.gpt.gpt_train import gpt_train
from transformer_model.common_model import Transformer
from transformer_model.models.gpt.gpt_utils import (
    BLOCK_SIZE,
    DEVICE,
    DROPOUT,
    LEARNING_RATE,
    NUM_EMBED,
    NUM_HEAD,
    NUM_LAYER,
    encode,
)

Number of patches (N) with image height (H=224), width (W=224) and patch size (P=16): 196
Input shape (single 2D image): (224, 224, 3)
Output shape (single 2D image flattened into patches): (196, 768)


In [ ]:
# raw data
path_do_data = "GPT_data/english.txt"
data_raw = open(path_do_data, encoding="utf-8").read()
# we use pretrained BERT tokenizer for performance improvements
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
vocab_size = tokenizer.vocab_size
# data_raw = data_raw[4000000:] # short dataset

# train/val split
data = encode(text_seq=data_raw, tokenizer=tokenizer)
n = int(0.9 * len(data))  # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# train a new model
gpt_model = Transformer(
    vocab_size=vocab_size,
    embed_size=NUM_EMBED,
    block_size=BLOCK_SIZE,
    n_heads=NUM_HEAD,
    num_layers=NUM_LAYER,
    dropout=DROPOUT,
    algorithm="GPT"
)
# load model to GPU if available
gpt_model = gpt_model.to(DEVICE)
# print the number of parameters in the model
print(
    "Model with {:.2f}M parameters".format(sum(p.numel() for p in gpt_model.parameters()) / 1e6)
)

Token indices sequence length is longer than the specified maximum sequence length for this model (37443 > 512). Running this sequence through the model will result in indexing errors


Model with 89.48M parameters


In [ ]:
# optimizer takes the model's parameters and the learning rate as input,
# and updates the parameters during the training process in order to
# minimize the loss function.
optimizer = torch.optim.AdamW(gpt_model.parameters(), lr=LEARNING_RATE)
MAX_ITER = 9991
gpt_model_train = gpt_train(MAX_ITER, train_data, val_data, optimizer, gpt_model)

Training GPT...
step          0 | train loss 10.7038 | val loss 10.7169
step        500 | train loss 0.4664 | val loss 8.3059
step       1000 | train loss 0.1703 | val loss 9.6975
step       1500 | train loss 0.1390 | val loss 10.2465
step       2000 | train loss 0.1283 | val loss 9.9672
step       2500 | train loss 0.1168 | val loss 10.5430
step       3000 | train loss 0.1114 | val loss 10.5862
step       3500 | train loss 0.1156 | val loss 10.9227
step       4000 | train loss 0.1095 | val loss 11.1550
step       4500 | train loss 0.1051 | val loss 11.2121
step       5000 | train loss 0.1101 | val loss 11.0840
step       5500 | train loss 0.1107 | val loss 11.3789
step       6000 | train loss 0.1074 | val loss 11.7482
step       6500 | train loss 0.1026 | val loss 11.8596
step       7000 | train loss 0.0988 | val loss 11.9320
step       7500 | train loss 0.0960 | val loss 12.4204
step       8000 | train loss 0.0984 | val loss 11.7259
step       8500 | train loss 0.0972 | val loss 12.0